In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from shared.constants import DatasetPath

In [3]:
DATASET = DatasetPath('ucidata-zachary')

In [4]:
spark = (SparkSession.builder
         .appName(f'{DATASET}_preprocess')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .getOrCreate())

22/01/01 21:35:25 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
22/01/01 21:35:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/01 21:35:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = (

    spark.read.csv(DATASET.raw('out.ucidata-zachary'), sep=' ')
        .withColumn('index', F.monotonically_increasing_id())
        .filter(F.col('index') > 1)
        .withColumnRenamed('_c0', 'src')
        .withColumnRenamed('_c1', 'dst')
)
df.head(5)

TypeError: path can be only string, list or RDD

In [ ]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

In [ ]:
df_edges = (
    df.select(F.col('src'), F.col('dst'))
        .distinct()
)
df_edges.head(5)

In [ ]:
df_nodes.write.parquet(DATASET.processed('nodes_Pupil'), mode='overwrite')
df_edges.write.parquet(DATASET.processed('nodes_TRAINS'), mode='overwrite')

In [ ]:
from datasets.build_schema import build_schema

build_schema(
    spark,
    name=str(DATASET),
    nodes=[
        ('Pupil', DATASET.processed('nodes_Pupil')),
    ],
    edges=[
        ('Trains', 'Pupil', 'Pupil', DATASET.processed('nodes_TRAINS')),
    ]
)